# 金融风控全链路能力展示

> This tutorial is only available in Chinese.

> Last updated: Nov 9, 2022
>
> 请使用v0.7.11或以上版本的隐语进行实验。
>
> 以下代码仅作为示例，请勿在生产环境直接使用。

本次实验将会展示如何使用隐语进行在风控领域常用的Logistic Regeression模型和XGB模型的模型研发工作。

隐语接下来将会开放模型部署和在线/离线模型预测功能，敬请期待。

## 实验目标

在本次实验中，我们将会利用一个开源数据集训练一个金融风控场景常用的线性回归和XGB模型。在此过程中将包含以下步骤：

- 样本对齐
- 特征预处理
- 数据分析
- 模型训练
- 模型预测
- 模型评估

请依次执行所有步骤确保实验可以顺利完成。

## 实验前置工作

### 初始化隐语框架

在本次实验中，我们将会包含两个节点：**alice** 和 **bob** . 在真实业务场景，他们将会代表两个不同实体，他们之间的原始数据不被允许直接相互传输，但是他们的原始数据将会被一起用以研发一个模型。

在下面的代码中，我们建立了一个 **SecretFlow Cluster**, 基于 **alice** 和 **bob** 两个节点，我们还创建了三个device：

- alice: PYU device, 负责在alice侧的本地计算，计算输入、计算过程和计算结果仅alice可见
- bob: PYU device, 负责在bob侧的本地计算，计算输入、计算过程和计算结果仅bob可见
- spu: SPU device, 负责alice和bob之间的密态计算，计算输入和计算结果为密态，由alice和bob各掌握一个分片，计算过程为MPC计算，由alice和bob各自的SPU Runtime一起执行。

>  如果你尚未理解以上的一些概念，比如SPU设备，请参考这篇 [文档](../developer/design/architecture.md).


In [1]:
import secretflow as sf

sf.shutdown()
sf.init(['alice', 'bob'], num_cpus=8, log_to_driver=True)
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))


/home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


在上面的log中，你应该发现，在**spu**的创建过程中，alice和bob两边都各有一个 **SPURuntime** 被建立并互相创建连接。

### 数据集

本次实验我们采用的原始数据是来自UCI的[Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing). 这个数据集汇集了一家葡萄牙银行机构电话营销的结果。

我们添加了**uid**这一列用于接下来隐私求交的实验。

我们首先看一下数据集所包含的信息。


In [2]:
import pandas as pd

# 当pd.dataframe过宽时，Sphinx目前的主题无法正确显示html格式输出，
# 如果自行运行notebook时可以忽略。
pd.set_option('display.notebook_repr_htm', False)

# secretflow.utils.simulation.datasets contains mirrors of some popular open dataset.
from secretflow.utils.simulation.datasets import dataset

df = pd.read_csv(dataset('bank_marketing_full'), sep=';')
df['uid'] = df.index + 1

df


       age           job   marital  education default  balance housing loan  \
0       58    management   married   tertiary      no     2143     yes   no   
1       44    technician    single  secondary      no       29     yes   no   
2       33  entrepreneur   married  secondary      no        2     yes  yes   
3       47   blue-collar   married    unknown      no     1506     yes   no   
4       33       unknown    single    unknown      no        1      no   no   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45206   51    technician   married   tertiary      no      825      no   no   
45207   71       retired  divorced    primary      no     1729      no   no   
45208   72       retired   married  secondary      no     5715      no   no   
45209   57   blue-collar   married  secondary      no      668      no   no   
45210   37  entrepreneur   married  secondary      no     2971      no   no   

         contact  day month  duration  campaign  pd

该数据集包含了45211个样本，每一个样本代表了一个目标客户。

每个样本包含16个feature，我们这里简单描述一下这个数据集所有的feature。


| feature | 描述 | 取值 |
| :-----| :---- | :---- |
| uid | 客户编码 | 数字 |
| age | 年龄 | 数字 |
| job | 工作类型 |  'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown' |
| marital | 婚姻状况 | 'divorced','married','single','unknown' |
| education | 教育状况 | 'tertiary', 'secondary', 'unknown', 'primary' |
| default | 是否有不良信用记录 | 'no','yes','unknown' |
| housing | 是否有房贷 |  'no','yes','unknown' |
| loan | 是否有个人贷款 | 'no','yes','unknown' |
| contact | 联系方式 | 'cellular','telephone' |
| month | 上次联系月份 | 'jan', 'feb', 'mar', ..., 'nov', 'dec' |
| day | 上次联系月日 |数字|
| duration | 上次沟通时间 | 数字 |
| campaign | 本次活动已经沟通的次数 | 数字 |
| pdays | 距离上次沟通经过的天数 | 数字 |
| previous | 在本次活动之前已经沟通的次数 | 数字 |
| poutcome | 之前活动的结果 | 'unknown', 'failure', 'other', 'success' | 




每个样本的label - y表示对于目标客户的营销结果（是否签订了定额存款合同），取值是'yes','no'。

我们假定以上16个feature由两个机构分别掌握，具体如下。

- alice: age, job, marital, education, default, balance, housing, loan
- bob: contact, day, month, duration, campaign, pdays, previous, poutcome, y


在真实业务场景中, alice和bob所掌握的数据可能是没有对齐的，为了模拟这种情况，我们将数据集shuffle之后，再随机各取90%来模拟这个状况。

In [3]:
import numpy as np

df_alice = df.iloc[:, np.r_[0:8, -1]].sample(frac=0.9)

df_alice


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


       age            job  marital  education default  balance housing loan  \
20060   48    blue-collar  married    primary      no      104      no   no   
16092   34     management  married   tertiary      no     1790     yes   no   
37583   35     technician  married   tertiary      no     -269     yes  yes   
39123   26    blue-collar   single  secondary      no      154     yes   no   
14785   51        retired  married    primary      no       34     yes   no   
...    ...            ...      ...        ...     ...      ...     ...  ...   
22513   35  self-employed  married  secondary      no     1256     yes   no   
14623   42    blue-collar  married  secondary      no        1      no   no   
14757   44         admin.  married  secondary      no      345     yes   no   
8866    28    blue-collar   single  secondary      no     1232     yes   no   
35507   31         admin.  married  secondary      no      799     yes   no   

         uid  
20060  20061  
16092  16093  
37583 

In [4]:
df_bob = df.iloc[:, 8:].sample(frac=0.9)

df_bob


         contact  day month  duration  campaign  pdays  previous poutcome  \
18487   cellular   31   jul        88        21     -1         0  unknown   
22171   cellular   21   aug       819        10     -1         0  unknown   
40960   cellular   13   aug       200         1    127         1  failure   
2884     unknown   14   may       339         1     -1         0  unknown   
29601   cellular    3   feb        88         5    251         3  failure   
...          ...  ...   ...       ...       ...    ...       ...      ...   
28063  telephone   28   jan        54         3    160        13  failure   
44531   cellular   12   aug       323         1    182         4  success   
20122   cellular   11   aug       856         3     -1         0  unknown   
34322   cellular    4   may       742         4     -1         0  unknown   
23400   cellular   27   aug        53         4     -1         0  unknown   

         y    uid  
18487   no  18488  
22171  yes  22172  
40960   no  409

我们这里将df_alice和df_bob保存为文件，作为alice和bob两方的原始输入。

至此，我们完成了所有实验准备工作。

In [5]:
import tempfile

_, alice_path = tempfile.mkstemp()
_, bob_path = tempfile.mkstemp()
df_alice.reset_index(drop=True).to_csv(alice_path, index=False)
df_bob.reset_index(drop=True).to_csv(bob_path, index=False)


## 样本对齐（隐私求交）

显然，第一步我们需要将两边的数据对齐。
隐私求交（[Private Set Intersection](https://en.wikipedia.org/wiki/Private_set_intersection))是一种密码学方法，可以获取两个集合的交集，而不泄露任何其他信息。
在隐语中，SPU设备支持三种隐私求交算法:

- [ECDH](https://ieeexplore.ieee.org/document/6234849/)：半诚实模型, 基于公钥密码学，原本适用于小数据集，但是隐语优化后已经能支持10亿量级的数据。
- [KKRT](https://eprint.iacr.org/2016/799.pdf)：半诚实模型, 基于布谷鸟哈希（Cuckoo Hashing）以及高效不经意传输扩展（OT Extension），适用于大数据集（比如千万数据集）。
- [BC22PCG](https://eprint.iacr.org/2022/334)：半诚实模型, 基于随机相关函数生成器，适用于大数据集。

由于我们这里的数据集较小，我们这里采用的是ECDH方法。


### 方式一：将隐私求交结果保存至文件

在一些应用场景场景中，alice和bob可能在隐私求交之后将结果直接保存至文件中，之后再进行后续操作。这个时候，请调用**psi_csv**接口。

在以下代码中，我们分别制定了两边需要求交的key以及输入和输出路径。

我们需要指定双方的输入文件和输出文件路径。对于ECDH来说，由于双方的地位是平等的，receiver并没有实际含义，你可以任意指定。我们需要设定正确的protocol。sort设为true之后，join的结果将会被排序。

> 请阅读 psi_csv 的文档。

In [6]:
_, alice_psi_path = tempfile.mkstemp()
_, bob_psi_path = tempfile.mkstemp()

spu.psi_csv(
    key="uid",
    input_path={alice: alice_path, bob: bob_path},
    output_path={alice: alice_psi_path, bob: bob_psi_path},
    receiver="alice",
    protocol="ECDH_PSI_2PC",
    sort=True,
)


(SPURuntime pid=115338) I1118 11:44:52.575792 115338 external/com_github_brpc_brpc/src/brpc/server.cpp:1070] Server[yasl::link::internal::ReceiverServiceImpl] is serving on port=48413.
(SPURuntime pid=115338) I1118 11:44:52.575893 115338 external/com_github_brpc_brpc/src/brpc/server.cpp:1073] Check out http://k69b13338.eu95sqa:48413 in web browser.
(SPURuntime pid=115337) I1118 11:44:52.573900 115337 external/com_github_brpc_brpc/src/brpc/server.cpp:1070] Server[yasl::link::internal::ReceiverServiceImpl] is serving on port=24103.
(SPURuntime pid=115337) I1118 11:44:52.573970 115337 external/com_github_brpc_brpc/src/brpc/server.cpp:1073] Check out http://k69b13338.eu95sqa:24103 in web browser.
(SPURuntime pid=115337) I1118 11:44:52.674812 115464 external/com_github_brpc_brpc/src/brpc/socket.cpp:2236] Checking Socket{id=0 addr=127.0.0.1:48413} (0x560058cdc4c0)
(SPURuntime pid=115337) I1118 11:44:52.675110 115462 external/com_github_brpc_brpc/src/brpc/socket.cpp:2296] Revived Socket{id=0 

(SPURuntime pid=115338) [2022-11-18 11:44:53.599] [info] [bucket_psi.cc:169] bucket size set to 1048576
(SPURuntime pid=115338) [2022-11-18 11:44:53.599] [info] [bucket_psi.cc:77] Begin sanity check for input file: /tmp/tmp57guvocf, precheck_switch:true
(SPURuntime pid=115338) [2022-11-18 11:44:53.621] [info] [csv_checker.cc:125] Executing duplicated scripts: LC_ALL=C sort --buffer-size=1G --temporary-directory=/tmp --stable selected-keys.1668743093599496476 | LC_ALL=C uniq -d > duplicate-keys.1668743093599496476
(SPURuntime pid=115338) [2022-11-18 11:44:53.644] [info] [bucket_psi.cc:90] End sanity check for input file: /tmp/tmp57guvocf, size=40690
(SPURuntime pid=115338) [2022-11-18 11:44:53.644] [info] [bucket_psi.cc:190] Run psi protocol=1, self_items_count=40690
(SPURuntime pid=115338) [2022-11-18 11:44:53.644] [info] [cryptor_selector.cc:50] Using libSodium
(SPURuntime pid=115338) [2022-11-18 11:44:53.644] [info] [cipher_store.cc:25] Disk cache choose num_bins=64
(SPURuntime pid=1

[{'party': 'alice', 'original_count': 40690, 'intersection_count': 36636},
 {'party': 'bob', 'original_count': 40690, 'intersection_count': 36636}]

### 方式二：将求交结果保存至VDataFrame

VDataFrame是隐语中保存垂直切分数据的数据结构，在接下来的任务中，我们将会不断使用VDataFrame的数据结构。

由于在本次实验中，经过隐私求交之后，我们还有后续操作，所以我们在这里使用 **data.vertical.read_csv** 来将原始数据隐私求交之后的结果直接转化为VDataFrame。

> 请阅读data.vertical.read_csv的文档。很多参数和psi_csv是一致的，这里不再赘述。

In [7]:
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv(
    {alice: alice_path, bob: bob_path},
    spu=spu,
    keys="uid",
    drop_keys="uid",
    psi_protocl="ECDH_PSI_2PC",
)
vdf.columns


(SPURuntime pid=115338) [2022-11-18 11:44:54.932] [info] [utils.cc:88] Finished sort scripts: tail -n +2 /tmp/tmp-sort-in-1668743094665280600 | LC_ALL=C sort --buffer-size=1G --temporary-directory=./ --stable --field-separator=, --key=10,10 >>/tmp/tmp-sort-out-1668743094665280600, ret=0
(SPURuntime pid=115338) [2022-11-18 11:44:54.932] [info] [bucket_psi.cc:157] End post filtering, in=/tmp/tmp57guvocf, out=/tmp/tmpc80n6brd
(SPURuntime pid=115338) [2022-11-18 11:44:57.128] [info] [bucket_psi.cc:169] bucket size set to 1048576
(SPURuntime pid=115338) [2022-11-18 11:44:57.128] [info] [bucket_psi.cc:77] Begin sanity check for input file: .data/1-4718d013-04b9-458b-a486-88288038543e/psi-input.csv, precheck_switch:true
(SPURuntime pid=115338) [2022-11-18 11:44:57.151] [info] [csv_checker.cc:125] Executing duplicated scripts: LC_ALL=C sort --buffer-size=1G --temporary-directory=.data/1-4718d013-04b9-458b-a486-88288038543e --stable selected-keys.1668743097128509212 | LC_ALL=C uniq -d > duplica

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

### 更多

我们在这里展示的是两方单键的隐私求交，隐语也支持三方和多键的隐私求交技术，想要了解更多信息，你可以：

- 阅读这篇[文档](https://spu.readthedocs.io/en/beta/development/psi.html)了解隐语SPU的隐私求交能力。
- 阅读该[教程](./PSI_On_SPU.ipynb)了解使用的例子。

## 特征预处理

一般情况下，我们都需要对用于建模的数据进行预处理，合理的预处理对模型训练效果非常关键。

在开始特征预处理之前，我们先使用 **stats.table_statistics.table_statistics** 来查看一下特征总体情况，我们会在后面专门讨论全表统计模块。

In [8]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats

          datatype  total_count  count  count_na     min       max  \
age          int64        36636  36636         0    18.0      95.0   
job         object        36636  36636         0     NaN       NaN   
marital     object        36636  36636         0     NaN       NaN   
education   object        36636  36636         0     NaN       NaN   
default     object        36636  36636         0     NaN       NaN   
balance      int64        36636  36636         0 -8019.0  102127.0   
housing     object        36636  36636         0     NaN       NaN   
loan        object        36636  36636         0     NaN       NaN   
contact     object        36636  36636         0     NaN       NaN   
day          int64        36636  36636         0     1.0      31.0   
month       object        36636  36636         0     NaN       NaN   
duration     int64        36636  36636         0     0.0    4918.0   
campaign     int64        36636  36636         0     1.0      63.0   
pdays        int64  

In [9]:
pd.reset_option('display.max_rows')

在接下来，我们将会展示隐语以下特征预处理能力：

- 值替换
- 缺失值填充
- WOE分组/分箱转换
- one-hot编码
- 标准化

### 值替换

我们先对以下特征做值替换：

| feature | 描述 | 取值和值替换规则 |
| :-----| :---- | :---- |
| education | 教育状况 | 'tertiary' -> 3, 'secondary' -> 2, 'unknown' -> 0, 'primary' -> 1 |
| default | 是否有不良信用记录 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| housing | 是否有房贷 |  'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| loan | 是否有个人贷款 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| month | 上次联系月份 | 'jan' -> 1, 'feb' -> 2, 'mar' -> 3, ..., 'nov' -> 11, 'dec' ->12 |
| y | label | 'yes' -> 1,'no' -> 0 |


替换完之后，我们使用 **sf.reveal** 来查看效果，请注意在生产中，**sf.reveal** 将会直接泄露数据，需要严格限制和进行审计。

> 在生产中，请严格限制**sf.reveal**的使用。

In [10]:
vdf['education'] = vdf['education'].replace(
    {'tertiary': 3, 'secondary': 2, 'primary': 1, 'unknown': np.NaN}
)

vdf['default'] = vdf['default'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['housing'] = vdf['housing'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['loan'] = vdf['loan'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['month'] = vdf['month'].replace(
    {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr': 4,
        'may': 5,
        'jun': 6,
        'jul': 7,
        'aug': 8,
        'sep': 9,
        'oct': 10,
        'nov': 11,
        'dec': 12,
    }
)

vdf['y'] = vdf['y'].replace(
    {
        'no': 0,
        'yes': 1,
    }
)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       46     management  married        3.0        0      229        1     0
3       33     technician   single        2.0        0       56        0     0
4       42     technician  married        2.0        0     8036        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36631   36  self-employed   single        3.0        0     4844        0     0
36632   49     technician  married        2.0        0      378        0     0
36633   49      housemaid  married        1.0        0     3376        0     0
36634   40    blue-collar  married        1.0        0       48        0     0
36635   46       services  married        3.0        0      474        0     0

[36636 rows x 8 columns]
       contact  day  month

#### 安全性讨论

值替换操作由数据所有者的PYU Device执行，不会泄露数据。

### 缺失值填充

接下来我们对缺失值进行填充。我们在这里均填充了众数，其他可选的策略还包括平均数、中位数等。

其他可能的处理方法包括删除缺省的行, 或者可以使用数据完整的行作为训练集，以此来预测缺失值。

替换完之后，我们使用 **sf.reveal** 来查看效果。

In [11]:
vdf["education"] = vdf["education"].fillna(vdf["education"].mode())
vdf["default"] = vdf["default"].fillna(vdf["default"].mode())
vdf["housing"] = vdf["housing"].fillna(vdf["housing"].mode())
vdf["loan"] = vdf["loan"].fillna(vdf["loan"].mode())

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       46     management  married        3.0        0      229        1     0
3       33     technician   single        2.0        0       56        0     0
4       42     technician  married        2.0        0     8036        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36631   36  self-employed   single        3.0        0     4844        0     0
36632   49     technician  married        2.0        0      378        0     0
36633   49      housemaid  married        1.0        0     3376        0     0
36634   40    blue-collar  married        1.0        0       48        0     0
36635   46       services  married        3.0        0      474        0     0

[36636 rows x 8 columns]
       contact  day  month

#### 安全性讨论

所填充的缺失值由属于数据所有者的PYU Device执行，并在接下来的缺失值操作中由数据所有者的PYU Device使用，不会泄露数据。

### woe分箱

woe分箱用于将连续值替换为离散值。

将连续型特征离散化的一个好处是可以有效地克服数据中隐藏的缺陷： 使模型结果更加稳定。例如，数据中的极端值是影响模型效果的一个重要因素。极端值导致模型参数过高或过低，或导致模型被虚假现象"迷惑"，把原来不存在的关系作为重要模式来学习。而离散化可以有效地减弱极端值和异常值的影响。

变量duration的75%分位数远小于最大值，而且该变量的标准差相对也比较大。因此需要对变量duration进行离散化。

In [12]:
from secretflow.preprocessing.binning.vert_woe_binning import VertWoeBinning
from secretflow.preprocessing.binning.vert_woe_substitution import VertWOESubstitution

binning = VertWoeBinning(spu)
woe_rules = binning.binning(
    vdf,
    binning_method="chimerge",
    bin_num=4,
    bin_names={alice: [], bob: ["duration"]},
    label_name="y",
)

woe_sub = VertWOESubstitution()
vdf = woe_sub.substitution(vdf, woe_rules)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       46     management  married        3.0        0      229        1     0
3       33     technician   single        2.0        0       56        0     0
4       42     technician  married        2.0        0     8036        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36631   36  self-employed   single        3.0        0     4844        0     0
36632   49     technician  married        2.0        0      378        0     0
36633   49      housemaid  married        1.0        0     3376        0     0
36634   40    blue-collar  married        1.0        0       48        0     0
36635   46       services  married        3.0        0      474        0     0

[36636 rows x 8 columns]
       contact  day  month

#### 安全性讨论

woe分桶需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### One Hot编码

one-hot编码适用于将类型编码转化为数值编码。 对于job、marital等特征我们需要one-hot编码。

In [13]:
from secretflow.preprocessing.encoder import OneHotEncoder

encoder = OneHotEncoder()
# for vif and correlation only
vdf_hat = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

tranformed_df = encoder.fit_transform(vdf['job'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['marital'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['contact'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['month'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['day'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['poutcome'])
vdf[tranformed_df.dtypes.index] = tranformed_df

vdf = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


       age  education  default  balance  housing  loan  job_admin.  \
0       58        3.0        0     2143        1     0         0.0   
1       43        2.0        0      593        1     0         0.0   
2       46        3.0        0      229        1     0         0.0   
3       33        2.0        0       56        0     0         0.0   
4       42        2.0        0     8036        0     0         0.0   
...    ...        ...      ...      ...      ...   ...         ...   
36631   36        3.0        0     4844        0     0         0.0   
36632   49        2.0        0      378        0     0         0.0   
36633   49        1.0        0     3376        0     0         0.0   
36634   40        1.0        0       48        0     0         0.0   
36635   46        3.0        0      474        0     0         0.0   

       job_blue-collar  job_entrepreneur  job_housemaid  ...  job_retired  \
0                  0.0               0.0            0.0  ...          0.0   
1    

#### 安全性讨论

one-hot编码操作由数据所有者的PYU Device执行，不会泄露数据。

### 标准化 
特征之间数值差距太大会使得模型收敛困难，我们一般先对数值进行标准化。

In [14]:
from secretflow.preprocessing import StandardScaler

X = vdf.drop(columns=['y'])
y = vdf['y']
scaler = StandardScaler()
X = scaler.fit_transform(X)
vdf[X.columns] = X
print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


(_run pid=115021) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=115021)   warnings.warn(
(_run pid=117122) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=117122)   warnings.warn(


            age  education   default   balance   housing      loan  \
0      1.608949   1.309163 -0.135655  0.269005  0.892221 -0.435471   
1      0.194862  -0.223577 -0.135655 -0.259179  0.892221 -0.435471   
2      0.477680   1.309163 -0.135655 -0.383217  0.892221 -0.435471   
3     -0.747862  -0.223577 -0.135655 -0.442169 -1.120798 -0.435471   
4      0.100590  -0.223577 -0.135655  2.277123 -1.120798 -0.435471   
...         ...        ...       ...       ...       ...       ...   
36631 -0.465045   1.309163 -0.135655  1.189406 -1.120798 -0.435471   
36632  0.760497  -0.223577 -0.135655 -0.332443 -1.120798 -0.435471   
36633  0.760497  -1.756317 -0.135655  0.689166 -1.120798 -0.435471   
36634 -0.087955  -1.756317 -0.135655 -0.444895 -1.120798 -0.435471   
36635  0.477680   1.309163 -0.135655 -0.299730 -1.120798 -0.435471   

       job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  ...  \
0        -0.35766        -0.524325         -0.183783      -0.166998  ...   
1      

#### 安全性讨论

标准化操作由数据所有者的PYU Device执行，不会泄露数据。

### 更多

隐语还支持其他更多的特征预处理能力，请参考这篇[文档](./data_preprocessing_with_data_frame.ipynb).

至此，我们已经完成了所有特征预处理工作。

> 本文主要目的是为了展示隐语的预处理能力，本文对于数据预处理方法的使用可能是有争议的，敬请谅解。

## 数据分析

在建模之前，我们有必要分析一下我们所使用的数据，以便确认是否需要重复特征预处理的过程。

下面我们将会展示隐语以下数据分析能力:

- 全表统计
- 相关系数矩阵
- VIF指标计算


### 全表统计

我们提供了类似于 **pd.DataFrame.describe** 来展示所有特征的基本统计信息。

> 在特征预处理的过程中，你可以不断调用全表统计来关注预处理效果。

In [15]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats


                  datatype  total_count  count  count_na       min  \
duration           float64        36636  36636         0 -2.846642   
campaign           float64        36636  36636         0 -0.567582   
pdays              float64        36636  36636         0 -0.412963   
previous           float64        36636  36636         0 -0.245282   
y                    int64        36636  36636         0  0.000000   
contact_cellular   float64        36636  36636         0 -1.365194   
contact_telephone  float64        36636  36636         0 -0.261625   
contact_unknown    float64        36636  36636         0 -0.631549   
month_1.0          float64        36636  36636         0 -0.179048   
month_2.0          float64        36636  36636         0 -0.250427   
month_3.0          float64        36636  36636         0 -0.103998   
month_4.0          float64        36636  36636         0 -0.263703   
month_5.0          float64        36636  36636         0 -0.661264   
month_6.0          f

In [16]:
pd.reset_option('display.max_rows')


#### 安全性讨论

请注意，全表统计会暴露数据整体统计结果，其背后实际上蕴含了**sf.reveal**,请谨慎使用。

### 相关系数矩阵

我们接下来计算特征和特征之间，特征和标签之间的相关系数矩阵。

> 计算相关系数矩阵时，one-hot编码各列无需参与计算。

In [17]:
from secretflow.stats.ss_pearsonr_v import PearsonR

pearson_r_calculator = PearsonR(spu)
corr_matrix = pearson_r_calculator.pearsonr(vdf_hat)

import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
corr_matrix


(_run pid=117122) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=117122)   warnings.warn(
(_run pid=117121) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=117121)   warnings.warn(


(_run pid=117122) [2022-11-18 11:45:14.038] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_run pid=117121) [2022-11-18 11:45:14.185] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(_run pid=117122) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=117121) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_spu_compile pid=115022) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


array([[1.000, -0.179, 0.016, 0.009, 0.307, -0.010, -0.004, -0.007,
        0.016, -0.004, -0.010],
       [-0.179, 1.000, -0.088, -0.031, -0.072, 0.004, 0.004, 0.017,
        -0.016, -0.024, 0.007],
       [0.016, -0.088, 1.000, 0.440, 0.103, -0.021, 0.003, -0.028, 0.003,
        0.124, -0.023],
       [0.009, -0.031, 0.440, 1.000, 0.089, 0.004, 0.027, -0.016, 0.017,
        0.036, -0.010],
       [0.307, -0.072, 0.103, 0.089, 1.000, 0.024, 0.073, -0.026, 0.052,
        -0.143, -0.071],
       [-0.010, 0.004, -0.021, 0.004, 0.024, 1.000, -0.163, -0.015,
        0.097, -0.183, -0.015],
       [-0.004, 0.004, 0.003, 0.027, 0.073, -0.163, 1.000, -0.010, 0.068,
        -0.073, -0.030],
       [-0.007, 0.017, -0.028, -0.016, -0.026, -0.015, -0.010, 1.000,
        -0.069, -0.009, 0.076],
       [0.016, -0.016, 0.003, 0.017, 0.052, 0.097, 0.068, -0.069, 1.000,
        -0.068, -0.085],
       [-0.004, -0.024, 0.124, 0.036, -0.143, -0.183, -0.073, -0.009,
        -0.068, 1.000, 0.040],
       

#### 安全性讨论

相关系数矩阵的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### VIF指标计算

隐语还支持VIF的计算来进行多重共线性检验。

> 计算VIF指标时，one-hot编码各列无需参与计算。

In [18]:
from secretflow.stats.ss_vif_v import VIF

vif_calculator = VIF(spu)
vif_results = vif_calculator.vif(vdf_hat)
print(vdf_hat.columns)
print(vif_results)


(_run pid=115022) [2022-11-18 11:45:14.679] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(_run pid=115022) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=115022)   warnings.warn(
(_run pid=117121) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=117121)   warnings.warn(


Index(['duration', 'campaign', 'pdays', 'previous', 'y', 'age', 'education',
       'default', 'balance', 'housing', 'loan'],
      dtype='object')
[1.139 1.042 1.277 1.244 1.160 1.080 1.051 1.010 1.031 1.091 1.018]


#### 安全性讨论

VIF指标的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

## 模型训练

接下来，我们将会分别训练一个逻辑回归模型和一个XGB模型。


### 随机分割

在训练之前，我们需要将数据分割为训练集和验证集。

其中train_x和train_y为训练集的特征和标签。test_x和test_y为训练集的特征和标签。


In [19]:
from secretflow.data.split import train_test_split

random_state = 1234

train_vdf, test_vdf = train_test_split(vdf, train_size=0.8, random_state=random_state)

train_x = train_vdf.drop(columns=['y'])
train_y = train_vdf['y']

test_x = test_vdf.drop(columns=['y'])
test_y = test_vdf['y']


#### 安全性讨论

随机分割时，每一方会共享随机数种子，并由每一方数据的owner分别执行各自的数据分割并且确保最终分割结果仍然是对齐的。

### PSI（人群稳定性分析）

样本稳定指数是衡量样本变化所产生的偏移量的一种重要指标，通常用来衡量样本的稳定程度，比如样本在两个月份之间的变化是否稳定。通常变量的PSI值在0.1以下表示变化不太显著，在0.1到0.25之间表示有比较显著的变化，大于0.25表示变量变化比较剧烈，需要特殊关注。

接下来以`balance`为例子，确认两次抽样的样本分布是否接近。

> 根据业务需求，PSI分析也可以在数据分析或者特征预处理的时候进行。


In [20]:
stats_df = table_statistics(train_x['balance'])

In [21]:
min_val, max_val = stats_df['min'], stats_df['max']

In [22]:
from secretflow.stats import psi_eval
from secretflow.stats.core.utils import equal_range
import jax.numpy as jnp

split_points = equal_range(jnp.array([min_val, max_val]), 3)
balance_psi_score = psi_eval(train_x['balance'], test_x['balance'], split_points)

sf.reveal(balance_psi_score)

DeviceArray(inf, dtype=float32)

#### 安全性讨论

PSI分析是一个单方运算，由数据owner的PYU Device执行计算。

### 逻辑回归模型

使用 **ml.linear.ss_sgd.SSRegression** 可以进行密态逻辑回归模型的训练。

请参考相关的API文档。


In [23]:
from secretflow.ml.linear.ss_sgd import SSRegression

lr_model = SSRegression(spu)
lr_model.fit(
    x=train_x,
    y=train_y,
    epochs=3,
    learning_rate=0.1,
    batch_size=1024,
    sig_type='t1',
    reg_type='logistic',
    penalty='l2',
    l2_norm=0.5,
)


(_run pid=115021) [2022-11-18 11:45:17.898] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_spu_compile pid=117121) /* error: missing value */
(_spu_compile pid=117121) {}


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(_run pid=119150) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


你可能会对为何上面的语句很快就执行完毕感到困惑，原因是在隐语中，语句都是lazy evaluation的，在上面的例子中，直到lr_model被真正被使用的时候，**lr_model.fit**才会被执行。

#### 安全性讨论

SSRegression的训练基于SPU Device，双方的原始数据将会被保护。

### XGBoost模型

使用 **ml.boost.ss_xgb_v.Xgb** 可以进行密态XGBoost模型的训练。

请参考相关的API文档。

In [24]:
from secretflow.ml.boost.ss_xgb_v import Xgb

xgb = Xgb(spu)
params = {
    'num_boost_round': 3,
    'max_depth': 5,
    'sketch_eps': 0.25,
    'objective': 'logistic',
    'reg_lambda': 0.2,
    'subsample': 1,
    'colsample_bytree': 1,
    'base_score': 0.5,
}
xgb_model = xgb.train(params=params, dtrain=train_x, label=train_y)


(_spu_compile pid=119150) /* error: missing value */
(_spu_compile pid=119150) {}


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(_run pid=119149) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


(_run pid=119150) [2022-11-18 11:45:29.490] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_spu_compile pid=119149) /* error: missing value */
(_spu_compile pid=119149) {}:task_name:_run


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

(_run pid=122576) [2022-11-18 11:46:17.095] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_spu_compile pid=122573) /* error: missing value */
(_spu_compile pid=122573) {}:task_name:_run


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

(_run pid=127951) [2022-11-18 11:47:06.449] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_spu_compile pid=127954) /* error: missing value */
(_spu_compile pid=127954) {}:task_name:_run


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

(scheduler +2m22s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +2m22s) Warning: The following resource request cannot be scheduled right now: {'bob': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +2m22s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'alice': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

Xgb.train将会直接执行，请耐心等待。

#### 安全性讨论

Xgb的训练基于SPU Device，双方的原始数据将会被保护。

## 模型预测

接下来，我们将会分别利用刚刚训练好的模型来预测测试集。

### 逻辑回归模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [25]:
lr_y_hat = lr_model.predict(x=test_x, batch_size=1024, to_pyu=bob)

#### 安全性讨论

逻辑回归的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

### XGBoost模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [26]:
xgb_y_hat = xgb_model.predict(dtrain=test_x, to_pyu=bob)


(_run pid=8610) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=8604) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


(_run pid=8610) [2022-11-18 11:47:57.073] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_run pid=8604) [2022-11-18 11:47:57.071] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported 

(_run pid=9177) [2022-11-18 11:47:59.395] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


#### 安全性讨论

XGBoost模型的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

## 模型评估

接下来，我们将利用测试数据集对模型效果进行评估，包括：

- 二分类评估
- PVA
- P-Value
- 评分卡转换

### 二分类评估

隐语中对二分类的评估有集成的支持。

`BiClassificationEval` 将计算 `AUC`, `KS`, `F1 Score`, `Lift`, `K-S`, `Gain`, `Precision`, `Recall` 等统计数值， 并提供（基于prediction score的）等频和等距分箱的统计报告和总报告。

不同分桶中评估模型的预测的`threshold`不同。总报告中依赖`threshold`的统计取的是各个分桶的最佳值。

详情可以参考API文档。

In [27]:
from secretflow.stats.biclassification_eval import BiClassificationEval

biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=lr_y_hat, bucket_size=20
)
lr_report = sf.reveal(biclassification_evaluator.get_all_reports())


In [28]:
print(f'positive_samples: {lr_report.summary_report.positive_samples}')
print(f'negative_samples: {lr_report.summary_report.negative_samples}')
print(f'total_samples: {lr_report.summary_report.total_samples}')
print(f'auc: {lr_report.summary_report.auc}')
print(f'ks: {lr_report.summary_report.ks}')
print(f'f1_score: {lr_report.summary_report.f1_score}')


positive_samples: 802.0
negative_samples: 6526.0
total_samples: 7328.0
auc: 0.8937802314758301
ks: 0.6474749445915222
f1_score: 0.5240405797958374


In [29]:
biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=xgb_y_hat, bucket_size=20
)
xgb_report = sf.reveal(biclassification_evaluator.get_all_reports())


In [30]:
print(f'positive_samples: {xgb_report.summary_report.positive_samples}')
print(f'negative_samples: {xgb_report.summary_report.negative_samples}')
print(f'total_samples: {xgb_report.summary_report.total_samples}')
print(f'auc: {xgb_report.summary_report.auc}')
print(f'ks: {xgb_report.summary_report.ks}')
print(f'f1_score: {xgb_report.summary_report.f1_score}')


positive_samples: 802.0
negative_samples: 6526.0
total_samples: 7328.0
auc: 0.8860154151916504
ks: 0.6287506818771362
f1_score: 0.5218542814254761


### PVA (预测和实际平均值比较)

结果由`abs(mean(Acutal) - mean(Prediction))`计算获得, 值越小越好。

In [31]:
from secretflow.stats import pva_eval

lr_pva_score = pva_eval(test_y, lr_y_hat, 1)

sf.reveal(lr_pva_score)


DeviceArray(0.058, dtype=float32)

In [32]:
xgb_pva_score = pva_eval(test_y, xgb_y_hat, 1)

sf.reveal(xgb_pva_score)


DeviceArray(0.076, dtype=float32)

### P-Value
双方可通过p-value的值来判断参数是否显著，即该自变量是否可以有效预测因变量的变异, 从而判定对应的解释变量是否应包括在模型中。


In [33]:
from secretflow.stats import SSPValue

model = lr_model.save_model()
sspv = SSPValue(spu)
pvalues = sspv.pvalues(test_x, test_y, model)

pvalues

(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(raylet) /home/fengjun.feng/miniconda3/envs/sf/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
(_run pid=19761) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=19762) WARNING:absl:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


(_run pid=19761) [2022-11-18 11:49:22.943] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_run pid=19762) [2022-11-18 11:49:23.059] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


array([0.000, 0.604, 0.815, 0.753, 0.981, 0.985, 0.975, 0.956, 0.996,
       0.754, 0.959, 0.985, 0.978, 0.972, 0.972, 0.837, 0.841, 0.983,
       0.851, 0.973, 0.993, 0.984, 0.989, 0.988, 0.988, 0.967, 0.991,
       0.995, 0.948, 0.989, 0.979, 0.974, 0.999, 0.979, 0.991, 0.966,
       0.999, 0.969, 0.974, 0.996, 0.989, 0.970, 0.991, 0.973, 0.994,
       0.966, 0.999, 0.980, 0.955, 0.992, 0.977, 0.990, 0.822, 0.978,
       0.762, 0.361, 0.783, 0.489, 0.009, 0.114, 0.998, 0.987, 0.976,
       0.988, 0.993, 0.968, 0.996, 0.994, 0.925, 0.996, 0.996, 0.999,
       0.998, 0.990, 0.988, 0.000])

### 评分卡转换

> 严格来说，评分卡转化是对预测结果的后续处理，并不属于模型评估。


我们将 `y = 1` 的概率设为`p`， `odds = p / (1 - p)`, 评分卡设定的分值刻度可以通过将分值表示为比率对数的线性表达式来定义，即可表示为下式：

`Score = A - B log(odds)`， A 和 B 是可以设定的常数。隐语中提供了评分卡转换功能，详情可以参考API文档。

In [34]:
from secretflow.stats import BiClassificationEval, ScoreCard

sc = ScoreCard(20, 600, 20)
score = sc.transform(xgb_y_hat)

sf.reveal(score.partitions[bob])


array([[452.095],
       [514.442],
       [499.259],
       ...,
       [470.005],
       [452.095],
       [515.080]])

### 安全性讨论

以上所有模型评估的方法均为单方运算，由label拥有者的PYU Device进行运算。

## 实验结束

最后，我们需要清理临时文件，并关闭隐语cluster。

In [35]:
import os

try:
    os.remove(alice_path)
    os.remove(alice_psi_path)
    os.remove(bob_path)
    os.remove(bob_psi_path)
except OSError:
    pass

sf.shutdown()

恭喜！你已经完成了隐语金融风控全链路的全部实验内容。

如果你对本实验有任何建议和问题，请在[Github Issues](https://github.com/secretflow/secretflow/issues)上联系我们。